In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tatanic-dataset/train(1).csv
/kaggle/input/tatanic-dataset/test.csv
/kaggle/input/tatanic-dataset/gender_submission.csv


In [2]:
import warnings 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib as plt
from sklearn.model_selection import cross_val_score
warnings.filterwarnings('ignore')

# # Loading Data

In [3]:
df_train = pd.read_csv("/kaggle/input/tatanic-dataset/train(1).csv")
df_train.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_test = pd.read_csv("/kaggle/input/tatanic-dataset/test.csv")
df_test.head()



,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df_gender = pd.read_csv("/kaggle/input/tatanic-dataset/gender_submission.csv")
df_gender.head()



,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# # EDA

Count Null Values

In [6]:
def find_null_values(data):
    null_columns = [(col, data[col].isnull().sum()) for col in data.columns if data[col].isnull().sum() != 0]
    return  pd.DataFrame(null_columns, columns=['Column', 'Null_Count'])


In [7]:
df = find_null_values(df_train)
df

,Column,Null_Count
0,Age,177
1,Cabin,687
2,Embarked,2


In [8]:
df = find_null_values(df_test)
df

,Column,Null_Count
0,Age,86
1,Fare,1
2,Cabin,327


In [9]:
 find_null_values(df_gender)

,Column,Null_Count


Find Duplicates Values

In [10]:
df_train.duplicated().sum()

0

In [11]:
df_test.duplicated().sum()

0

In [12]:
df_gender.duplicated().sum()

0

Statical Analysis

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [14]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [15]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [16]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,418.0,1100.500000,120.810458,892.00,996.2500,1100.5000,1204.75,1309.0000
Pclass,418.0,2.265550,0.841838,1.00,1.0000,3.0000,3.00,3.0000
Age,332.0,30.272590,14.181209,0.17,21.0000,27.0000,39.00,76.0000
SibSp,418.0,0.447368,0.896760,0.00,0.0000,0.0000,1.00,8.0000
Parch,418.0,0.392344,0.981429,0.00,0.0000,0.0000,0.00,9.0000
Fare,417.0,35.627188,55.907576,0.00,7.8958,14.4542,31.50,512.3292


In [17]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [18]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [19]:
#i dont wanna delete issing values i will replace them by the most common
def fill_missing_values(df):
    for column in df.columns:
        if df[column].dtype == 'object':
            most_common_value = df[column].mode()[0]
            df[column].fillna(most_common_value, inplace=True)
        else:
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
            
    return df

In [20]:
fill_missing_values(df_train)
df_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [21]:
fill_missing_values(df_test)
df_test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

#  Outlier

In [22]:
from collections import Counter 
def detect_outline(df,features):
    outlier_indeces=[]
    for c in features:
        Q1= np.percentile(df[c],25)
        Q3= np.percentile(df[c],75)
        IQR=Q3-Q1
        outlier_step=IQR*1.5
        outlier_list=df[(df[c]<Q1 - outlier_step) | (df[c]>Q3+outlier_step)].index
        outlier_indeces.extend(outlier_list)
    outlier_indeces=Counter(outlier_indeces)
    multiple_outlier=list(i for i,v in outlier_indeces.items()if v>2 )
    return multiple_outlier

In [23]:
df_train.loc[detect_outline(df_train,["Age","SibSp","Fare","Parch"])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,B96 B98,S
16,17,0,3,"Rice, Master. Eugene",male,2.000000,4,1,382652,29.1250,B96 B98,Q
119,120,0,3,"Andersson, Miss. Ellis Anna Maria",female,2.000000,4,2,347082,31.2750,B96 B98,S
164,165,0,3,"Panula, Master. Eino Viljami",male,1.000000,4,1,3101295,39.6875,B96 B98,S
268,269,1,1,"Graham, Mrs. William Thompson (Edith Junkins)",female,58.000000,0,1,PC 17582,153.4625,C125,S
297,298,0,1,"Allison, Miss. Helen Loraine",female,2.000000,1,2,113781,151.5500,C22 C26,S
305,306,1,1,"Allison, Master. Hudson Trevor",male,0.920000,1,2,113781,151.5500,C22 C26,S
386,387,0,3,"Goodwin, Master. Sidney Leonard",male,1.000000,5,2,CA 2144,46.9000,B96 B98,S
438,439,0,1,"Fortune, Mr. Mark",male,64.000000,1,4,19950,263.0000,C23 C25 C27,S
587,588,1,1,"Frolicher-Stehli, Mr. Maxmillian",male,60.000000,1,1,13567,79.2000,B41,C


In [24]:
df_train.drop(detect_outline(df_train,["Age","SibSp","Fare","Parch"]),axis=0,inplace=True)
df_train.PassengerId[28]

29

In [25]:
df_train_len = len(df_train)
df_train=pd.concat([df_train,df_test],axis=0).reset_index(drop = True)
df_train[df_train.Embarked.isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [26]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.00000,1,0,A/5 21171,7.2500,B96 B98,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00000,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.00000,0,0,STON/O2. 3101282,7.9250,B96 B98,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00000,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.00000,0,0,373450,8.0500,B96 B98,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1279,1305,NaN,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,S
1280,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C
1281,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,S
1282,1308,NaN,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,B57 B59 B63 B66,S


In [27]:
df_train.isna().sum()

PassengerId      0
Survived       418
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

In [28]:
df_train.Embarked=df_train.Embarked.fillna("C")
df_train[df_train.Embarked.isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [29]:
df_train[df_train.Fare.isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [30]:
df_train[df_train.Pclass==3]["Fare"].mean()

12.563413508617568

In [31]:
df_train.Fare=df_train.Fare.fillna(12)
df_train[df_train.Fare.isnull()]
df_train[df_train.PassengerId==1044]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1018,1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,35.627188,B57 B59 B63 B66,S


In [32]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.00000,1,0,A/5 21171,7.2500,B96 B98,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00000,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.00000,0,0,STON/O2. 3101282,7.9250,B96 B98,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00000,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.00000,0,0,373450,8.0500,B96 B98,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1279,1305,NaN,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,S
1280,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C
1281,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,S
1282,1308,NaN,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,B57 B59 B63 B66,S


# Visualisation

In [33]:
df

,Column,Null_Count
0,Age,86
1,Fare,1
2,Cabin,327


In [34]:
df=df_train
survived_counts = df['Survived'].value_counts()
fig = px.bar(x=survived_counts.index, y=survived_counts.values, color=survived_counts.index,
             labels={'x': 'Survived', 'y': 'Count', 'color': 'Survived'},
             title='Survival Status of Passengers')


for i, count in enumerate(survived_counts.values):
    fig.add_annotation(x=i, y=count, text=str(count), showarrow=True, arrowhead=1)

fig.show()

In [35]:

survived_counts = df['Sex'].value_counts()
fig = px.bar(x=survived_counts.index, y=survived_counts.values, color=survived_counts.index,
             labels={'x': 'Survived', 'y': 'Count', 'color': 'Survived'},
             title='Survival Status of Passengers')


for i, count in enumerate(survived_counts.values):
    fig.add_annotation(x=i, y=count, text=str(count), showarrow=True, arrowhead=1)

fig.show()

In [36]:
fig = px.histogram(df, x='Age', color='Survived', nbins=20,
                   labels={'Age': 'Age', 'Personnes': 'Count', 'Survived': 'Survived'},
                   title='Distribution of Age by Survival Status')
fig.show()

In [37]:
fig = px.histogram(df, x='Pclass', color='Survived',
                   labels={'Pclass': 'Passenger Class', 'count': 'Count', 'Survived': 'Survived'},
                   title='Survival Status by Passenger Class')
fig.show()

In [38]:
fig = px.histogram(df, x='Parch', color='Survived',
                   labels={'Parch': 'Number of Parents/Children', 'count': 'Count', 'Survived': 'Survived'},
                   title='Survival Status by Number of Parents/Children')
fig.show()

In [39]:
fig = px.histogram(df, x='SibSp', color='Survived',
                   labels={'Parch': 'Number of Parents/Children', 'count': 'Count', 'Survived': 'Survived'},
                   title='Survival Status by Number of siblings / spouses ')
fig.show()

# Features Engeneering

In [40]:
df_train.dtypes

PassengerId      int64
Survived       float64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [41]:
df_train.Name.head(10)

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
5                                     Moran, Mr. James
6                              McCarthy, Mr. Timothy J
7    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
8                  Nasser, Mrs. Nicholas (Adele Achem)
9                      Sandstrom, Miss. Marguerite Rut
Name: Name, dtype: object

In [42]:
name=df_train.Name
df_train["Title"]=[i.split(".")[0].split(",")[-1].strip() for i in name]
df_train.Title.head(10)

0      Mr
1     Mrs
2    Miss
3     Mrs
4      Mr
5      Mr
6      Mr
7     Mrs
8     Mrs
9    Miss
Name: Title, dtype: object

In [43]:
df_train.Title=df_train["Title"].replace(["Don","Rev","Dr","Major","Lady","Sir","Col","Capt","the Countess","Jonkheer"],"Other")
df_train.Title=[0 if i =="Master" else 1 if i=="Mrs" or i=="Miss" or i=="Ms" or i=="Mlle" else 2 if i=="Mr" else 3 for i in df_train.Title]
df_train["Title"].head(20)

0     2
1     1
2     1
3     1
4     2
5     2
6     2
7     1
8     1
9     1
10    1
11    2
12    2
13    1
14    1
15    2
16    1
17    1
18    2
19    2
Name: Title, dtype: int64

In [44]:
df_train.drop(["Name"], axis = 1, inplace = True)
df_train.head(10)


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0.0,3,male,22.000000,1,0,A/5 21171,7.2500,B96 B98,S,2
1,2,1.0,1,female,38.000000,1,0,PC 17599,71.2833,C85,C,1
2,3,1.0,3,female,26.000000,0,0,STON/O2. 3101282,7.9250,B96 B98,S,1
3,4,1.0,1,female,35.000000,1,0,113803,53.1000,C123,S,1
4,5,0.0,3,male,35.000000,0,0,373450,8.0500,B96 B98,S,2
5,6,0.0,3,male,29.699118,0,0,330877,8.4583,B96 B98,Q,2
6,7,0.0,1,male,54.000000,0,0,17463,51.8625,E46,S,2
7,9,1.0,3,female,27.000000,0,2,347742,11.1333,B96 B98,S,1
8,10,1.0,2,female,14.000000,1,0,237736,30.0708,B96 B98,C,1
9,11,1.0,3,female,4.000000,1,1,PP 9549,16.7000,G6,S,1


In [45]:
df_train["Fsize"]=df_train.SibSp+df_train.Parch
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Fsize
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,B96 B98,S,2,1
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,1,1
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S,1,0
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,1,1
4,5,0.0,3,male,35.0,0,0,373450,8.0500,B96 B98,S,2,0


In [46]:
df_train = pd.get_dummies(df_train,columns=["Title"],dtype=int)
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fsize,Title_0,Title_1,Title_2,Title_3
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,B96 B98,S,1,0,0,1,0
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,1,0,1,0,0
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S,0,0,1,0,0
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,1,0,1,0,0
4,5,0.0,3,male,35.0,0,0,373450,8.0500,B96 B98,S,0,0,0,1,0


In [47]:
df_train["Fsize"]=df_train.SibSp+df_train.Parch
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fsize,Title_0,Title_1,Title_2,Title_3
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,B96 B98,S,1,0,0,1,0
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,1,0,1,0,0
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S,0,0,1,0,0
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,1,0,1,0,0
4,5,0.0,3,male,35.0,0,0,373450,8.0500,B96 B98,S,0,0,0,1,0


In [48]:
df_train["family_size"]=[1 if i<3.5 and i>0.5 else 0 for i in df_train.Fsize]
df_train.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fsize,Title_0,Title_1,Title_2,Title_3,family_size
0,1,0.0,3,male,22.000000,1,0,A/5 21171,7.2500,B96 B98,S,1,0,0,1,0,1
1,2,1.0,1,female,38.000000,1,0,PC 17599,71.2833,C85,C,1,0,1,0,0,1
2,3,1.0,3,female,26.000000,0,0,STON/O2. 3101282,7.9250,B96 B98,S,0,0,1,0,0,0
3,4,1.0,1,female,35.000000,1,0,113803,53.1000,C123,S,1,0,1,0,0,1
4,5,0.0,3,male,35.000000,0,0,373450,8.0500,B96 B98,S,0,0,0,1,0,0
5,6,0.0,3,male,29.699118,0,0,330877,8.4583,B96 B98,Q,0,0,0,1,0,0
6,7,0.0,1,male,54.000000,0,0,17463,51.8625,E46,S,0,0,0,1,0,0
7,9,1.0,3,female,27.000000,0,2,347742,11.1333,B96 B98,S,2,0,1,0,0,1
8,10,1.0,2,female,14.000000,1,0,237736,30.0708,B96 B98,C,1,0,1,0,0,1
9,11,1.0,3,female,4.000000,1,1,PP 9549,16.7000,G6,S,2,0,1,0,0,1


In [49]:
df_train = pd.get_dummies(df_train,columns=["family_size"],dtype=int)
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Fsize,Title_0,Title_1,Title_2,Title_3,family_size_0,family_size_1
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,B96 B98,S,1,0,0,1,0,0,1
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,1,0,1,0,0,0,1
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S,0,0,1,0,0,1,0
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,1,0,1,0,0,0,1
4,5,0.0,3,male,35.0,0,0,373450,8.0500,B96 B98,S,0,0,0,1,0,1,0


In [50]:
df_train.drop("Fsize",axis=1,inplace=True)
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title_0,Title_1,Title_2,Title_3,family_size_0,family_size_1
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,B96 B98,S,0,0,1,0,0,1
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,0,1,0,0,0,1
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,B96 B98,S,0,1,0,0,1,0
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,0,1,0,0,0,1
4,5,0.0,3,male,35.0,0,0,373450,8.0500,B96 B98,S,0,0,1,0,1,0


In [51]:
df_train.drop("SibSp",axis=1,inplace=True)
df_train.drop("Parch",axis=1,inplace=True)

In [52]:
df_train = pd.get_dummies(df_train,columns=["Embarked"],dtype=int)
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Cabin,Title_0,Title_1,Title_2,Title_3,family_size_0,family_size_1,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,male,22.0,A/5 21171,7.2500,B96 B98,0,0,1,0,0,1,0,0,1
1,2,1.0,1,female,38.0,PC 17599,71.2833,C85,0,1,0,0,0,1,1,0,0
2,3,1.0,3,female,26.0,STON/O2. 3101282,7.9250,B96 B98,0,1,0,0,1,0,0,0,1
3,4,1.0,1,female,35.0,113803,53.1000,C123,0,1,0,0,0,1,0,0,1
4,5,0.0,3,male,35.0,373450,8.0500,B96 B98,0,0,1,0,1,0,0,0,1


In [53]:
tickets=[]
for i in list(df_train.Ticket):
    if i.isdigit():
        tickets.append("x")
    else:
        tickets.append(i.replace(".","").replace("/","").strip().split(" ")[0])
df_train["Ticket"]=tickets
df_train.Ticket.head(20)

0         A5
1         PC
2     STONO2
3          x
4          x
5          x
6          x
7          x
8          x
9         PP
10         x
11        A5
12         x
13         x
14         x
15         x
16         x
17         x
18         x
19         x
Name: Ticket, dtype: object

In [54]:

df_train = pd.get_dummies(df_train,columns=["Ticket"],dtype=int,prefix="T")
df_train.head()


,PassengerId,Survived,Pclass,Sex,Age,Fare,Cabin,Title_0,Title_1,Title_2,...,T_SOTONO2,T_SOTONOQ,T_SP,T_STONO,T_STONO2,T_STONOQ,T_SWPP,T_WC,T_WEP,T_x
0,1,0.0,3,male,22.0,7.2500,B96 B98,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,1.0,1,female,38.0,71.2833,C85,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1.0,3,female,26.0,7.9250,B96 B98,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,4,1.0,1,female,35.0,53.1000,C123,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0.0,3,male,35.0,8.0500,B96 B98,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [55]:
df_train = pd.get_dummies(df_train,columns=["Pclass"],dtype=int)
df_train.head()

,PassengerId,Survived,Sex,Age,Fare,Cabin,Title_0,Title_1,Title_2,Title_3,...,T_STONO,T_STONO2,T_STONOQ,T_SWPP,T_WC,T_WEP,T_x,Pclass_1,Pclass_2,Pclass_3
0,1,0.0,male,22.0,7.2500,B96 B98,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1.0,female,38.0,71.2833,C85,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,1.0,female,26.0,7.9250,B96 B98,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
3,4,1.0,female,35.0,53.1000,C123,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
4,5,0.0,male,35.0,8.0500,B96 B98,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1


In [56]:
df_train = pd.get_dummies(df_train,columns=["Sex"],dtype=int)
df_train.head()

,PassengerId,Survived,Age,Fare,Cabin,Title_0,Title_1,Title_2,Title_3,family_size_0,...,T_STONOQ,T_SWPP,T_WC,T_WEP,T_x,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,1,0.0,22.0,7.2500,B96 B98,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
1,2,1.0,38.0,71.2833,C85,0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,3,1.0,26.0,7.9250,B96 B98,0,1,0,0,1,...,0,0,0,0,0,0,0,1,1,0
3,4,1.0,35.0,53.1000,C123,0,1,0,0,0,...,0,0,0,0,1,1,0,0,1,0
4,5,0.0,35.0,8.0500,B96 B98,0,0,1,0,1,...,0,0,0,0,1,0,0,1,0,1


In [57]:
PassengerId_df = pd.DataFrame({'PassengerId': df_train.iloc[:, 0]})
df_train.drop(["PassengerId","Cabin"],axis=1,inplace=True)
df_train.head()

,Survived,Age,Fare,Title_0,Title_1,Title_2,Title_3,family_size_0,family_size_1,Embarked_C,...,T_STONOQ,T_SWPP,T_WC,T_WEP,T_x,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,0.0,22.0,7.2500,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
1,1.0,38.0,71.2833,0,1,0,0,0,1,1,...,0,0,0,0,0,1,0,0,1,0
2,1.0,26.0,7.9250,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
3,1.0,35.0,53.1000,0,1,0,0,0,1,0,...,0,0,0,0,1,1,0,0,1,0
4,0.0,35.0,8.0500,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,1


# Modeling 

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [69]:
train = df_train[:df_train_len]
X_train = train.drop(labels = "Survived", axis = 1)
df_test =X_train
y_train = train["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.33, random_state = 42)

In [70]:
rf_classifier = RandomForestClassifier(random_state=42, 
                                        n_estimators=300, 
                                        max_features=10, 
                                        min_samples_split=3, 
                                        min_samples_leaf=10, 
                                        bootstrap=False, 
                                        criterion='gini',
                                        class_weight='balanced')


rf_classifier.fit(X_train, y_train)

cv_precision = cross_val_score(rf_classifier, X_train, y_train, cv=5, scoring='precision')
y_pred = rf_classifier.predict(X_test)


precision = precision_score(y_test, y_pred)
print("Cross-Validation Precision Scores:", cv_precision)
print("Average Cross-Validation Precision:", cv_precision.mean())
print("Test Set Precision Score:", precision)



Cross-Validation Precision Scores: [0.75       0.7        0.74       0.81578947 0.74      ]
Average Cross-Validation Precision: 0.749157894736842
Test Set Precision Score: 0.7380952380952381


In [71]:
PassengerId_df

,PassengerId
0,1
1,2
2,3
3,4
4,5
...,...
1279,1280
1280,1281
1281,1282
1282,1283


In [74]:
predictions = rf_classifier.predict(df_test)
PassengerId_df = pd.DataFrame({'PassengerId': range(1, len(df_test) + 1)})
output = pd.DataFrame({'PassengerId': PassengerId_df['PassengerId'], 'Survived': predictions})
output.to_csv('predictions.csv', index=False)
output

,PassengerId,Survived
0,1,0.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,0.0
...,...,...
861,862,0.0
862,863,1.0
863,864,1.0
864,865,1.0
